In [2]:
import pandas as pd
import numpy as np

In [11]:
from sklearn.decomposition import TruncatedSVD 

# Models Packages
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn import feature_selection
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

# Gradient Boosting
# import lightgbm as lgb
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold as kfold

# Tf-Idf
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import FeatureUnion
from sklearn import linear_model
from scipy.sparse import vstack, hstack, csr_matrix, load_npz
from nltk.corpus import stopwords 
# from Levenshtein import  jaro_winkler

# Viz
import seaborn as sns
import matplotlib.pyplot as plt
import re
import string

In [31]:
# training = pd.read_csv('../hasil/5_train_data_1000_25_06122023.csv', index_col = "item_id", parse_dates = ["activation_date"])
training = pd.read_csv('../hasil/traindata_lexicon.csv')
# traindex = training.index
testing = pd.read_csv('../hasil/testdata_lexicon.csv')
# testdex = testing.index


In [32]:
training.head()

,clean_content
0,Bagus sekali pemandangan hari ini
1,aku bisa makan pagi enak kenyang mantab sehat
2,makanan ini sangat enak lezat mantab aku suka ...
3,aku sangat menginginkan ini jadi aku akan kemb...
4,saat ini aku sangat kesal dan sangat capek


In [33]:
spath = 'sentilex-id/'
# sentiment_id_n = pd.read_csv(spath + 'negative_words_id.txt').rename(columns={'время':'Words'})
sentiment_id_n = pd.read_csv(spath + 'negative_words_id.txt')
sentiment_id_n['Polarity'] = -1
sentiment_id_p = pd.read_csv(spath + 'positive_words_id.txt')
sentiment_id_p['Polarity'] = 1
sentiment_id = pd.concat([sentiment_id_n, sentiment_id_p], ignore_index=True)
sentiment_id_vect = CountVectorizer()
sentiment_id2 = sentiment_id_vect.fit_transform(sentiment_id['Words'])
sentiment_id_model = linear_model.LinearRegression(n_jobs=-1).fit(sentiment_id2, sentiment_id['Polarity'])

In [34]:
print(sentiment_id_model)

LinearRegression(n_jobs=-1)


In [35]:
def getSentiment(s):
    return sentiment_id_model.predict(sentiment_id_vect.transform([s]))[0]

In [36]:
senti_cols = ['clean_content']
for c in senti_cols:
    training[c + '_Pol'] = training[c].map(lambda x: getSentiment(str(x))) #getSentiment
    testing[c + '_Pol'] = testing[c].map(lambda x: getSentiment(str(x))) #getSentiment

In [38]:
# print(training[:1])
training.head(25)

,clean_content,clean_content_Pol
0,Bagus sekali pemandangan hari ini,0.258621
1,aku bisa makan pagi enak kenyang mantab sehat,1.517241
2,makanan ini sangat enak lezat mantab aku suka ...,4.034483
3,aku sangat menginginkan ini jadi aku akan kemb...,0.258621
4,saat ini aku sangat kesal dan sangat capek,-1.000000
5,orang ini sangat menyebalkan karena dia telah ...,-3.224138
6,aku anak sehat tubuhku kuat karena ibuku rajin...,6.034483


In [57]:
df = pd.concat([training,testing],axis=0)

In [59]:
df.head(20)

,clean_content,clean_content_Pol
0,Bagus sekali pemandangan hari ini,0.258621
1,aku bisa makan pagi enak kenyang mantab sehat,1.517241
2,makanan ini sangat enak lezat mantab aku suka ...,4.034483
3,aku sangat menginginkan ini jadi aku akan kemb...,0.258621
4,saat ini aku sangat kesal dan sangat capek,-1.000000
5,orang ini sangat menyebalkan karena dia telah ...,-3.224138
6,aku anak sehat tubuhku kuat karena ibuku rajin...,6.034483
0,Bagus sekali pemandangan hari ini,0.258621
1,aku bisa makan pagi enak kenyang mantab sehat,1.517241
2,makanan ini sangat enak lezat mantab aku suka ...,4.034483


In [60]:
print('\nAll Data shape: {} Rows, {} Columns'.format(*df.shape))


All Data shape: 14 Rows, 2 Columns


In [61]:
df['clean_punc'] = df['clean_content'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))


In [62]:
df.head()

,clean_content,clean_content_Pol,clean_punc
0,Bagus sekali pemandangan hari ini,0.258621,0
1,aku bisa makan pagi enak kenyang mantab sehat,1.517241,0
2,makanan ini sangat enak lezat mantab aku suka ...,4.034483,0
3,aku sangat menginginkan ini jadi aku akan kemb...,0.258621,0
4,saat ini aku sangat kesal dan sangat capek,-1.000000,0


## Feature Engineering
Menambahkan fitur selain TFIDF

In [63]:
# Feature Engineering 

# Meta Text Features
textfeats = ["clean_content"]

In [64]:
for cols in textfeats:
    df[cols] = df[cols].astype(str) 
    df[cols] = df[cols].astype(str).fillna('missing') # FILL NA
    df[cols] = df[cols].str.lower() # Lowercase all text, so that capitalized words dont get treated differently
    df[cols + '_num_words'] = df[cols].apply(lambda comment: len(comment.split())) # Count number of Words
    df[cols + '_num_unique_words'] = df[cols].apply(lambda comment: len(set(w for w in comment.split())))
    df[cols + '_words_vs_unique'] = df[cols+'_num_unique_words'] / df[cols+'_num_words'] * 100 # Count Unique Words
    df[cols + '_num_letters'] = df[cols].apply(lambda comment: len(comment)) # Count number of Letters

# Extra Feature Engineering
# df['title_desc_len_ratio'] = df['title_num_letters']/df['description_num_letters'

In [65]:
df.head()

,clean_content,clean_content_Pol,clean_punc,clean_content_num_words,clean_content_num_unique_words,clean_content_words_vs_unique,clean_content_num_letters
0,bagus sekali pemandangan hari ini,0.258621,0,5,5,100.000000,33
1,aku bisa makan pagi enak kenyang mantab sehat,1.517241,0,8,8,100.000000,45
2,makanan ini sangat enak lezat mantab aku suka ...,4.034483,0,9,9,100.000000,52
3,aku sangat menginginkan ini jadi aku akan kemb...,0.258621,0,13,12,92.307692,73
4,saat ini aku sangat kesal dan sangat capek,-1.000000,0,8,7,87.500000,43


In [86]:
print("\n[TF-IDF] Term Frequency Inverse Document Frequency Stage")
id_stop = set(stopwords.words('indonesian'))


[TF-IDF] Term Frequency Inverse Document Frequency Stage


In [68]:
tfidf_para = {
    "stop_words": id_stop,
    "analyzer": 'word',
    "token_pattern": r'\w{1,}',
    "sublinear_tf": True,
    "dtype": np.float32,
    "norm": 'l2',
    #"min_df":5,
    #"max_df":.9,
    "smooth_idf":False
}


In [70]:
def get_col(col_name): return lambda x: x[col_name]

In [82]:
vectorizer = FeatureUnion([
        ('clean_content',TfidfVectorizer(
            ngram_range=(1, 2),
            max_features=17000,
            **tfidf_para,
            preprocessor=get_col('clean_content'))),
        # ('title',CountVectorizer(
        #     ngram_range=(1, 2),
        #     stop_words = russian_stop,
        #     max_features=130000,
        #     preprocessor=get_col('title')))
    ])

In [83]:
vectorizer

FeatureUnion(transformer_list=[('clean_content',
                                TfidfVectorizer(dtype=<class 'numpy.float32'>,
                                                max_features=17000,
                                                ngram_range=(1, 2),
                                                preprocessor=<function get_col.<locals>.<lambda> at 0x000001EF697A76D0>,
                                                smooth_idf=False,
                                                stop_words={'ada', 'adalah',
                                                            'adanya', 'adapun',
                                                            'agak', 'agaknya',
                                                            'agar', 'akan',
                                                            'akankah', 'akhir',
                                                            'akhiri',
                                                            'akhirnya', 'aku',
                                                            'akulah', 'amat',
                                                            'amatlah', 'anda',
                                                            'andalah', 'antar',
                                                            'antara',
                                                            'antaranya', 'apa',
                                                            'apaan', 'apabila',
                                                            'apakah', 'apalagi',
                                                            'apatah', 'artinya',
                                                            'asal', 'asalkan', ...},
                                                sublinear_tf=True,
                                                token_pattern='\\w{1,}'))])

In [84]:
#Fit my vectorizer on the entire dataset instead of the training rows
#Score improved by .0001
# df.to_dict
vectorizer.fit(df.to_dict())

InvalidParameterError: The 'stop_words' parameter of TfidfVectorizer must be a str among {'english'}, an instance of 'list' or None. Got {'bukan', 'kan', 'tampak', 'bila', 'tandas', 'terlihat', 'jangan', 'apalagi', 'termasuk', 'diibaratkan', 'sesudahnya', 'bersama', 'selalu', 'tersampaikan', 'tentunya', 'diperbuatnya', 'para', 'lah', 'tahu', 'keinginan', 'sepantasnyalah', 'pasti', 'memastikan', 'bawah', 'teringat', 'sudahkah', 'kelima', 'tutur', 'mengibaratkannya', 'memungkinkan', 'dimaksudkan', 'disinilah', 'sejenak', 'mengatakan', 'seolah-olah', 'tentu', 'bekerja', 'mirip', 'menanyai', 'walaupun', 'seringnya', 'bahwa', 'kembali', 'setempat', 'bagaikan', 'nyatanya', 'diminta', 'sekiranya', 'kapan', 'ketika', 'mana', 'itulah', 'diperlukannya', 'berturut', 'kesampaian', 'bolehkah', 'ujarnya', 'kiranya', 'disebutkannya', 'sela', 'tiba-tiba', 'entahlah', 'siap', 'kebetulan', 'sekurangnya', 'tadinya', 'sesampai', 'menanyakan', 'padahal', 'tentang', 'mengetahui', 'secara', 'sana', 'disampai